# **Forecast créditos generales**

In [10]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "financiamientos" in nombre and "tulancingo" in nombre
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_financiamientos_2019_tulancingo',
 'df_financiamientos_2020_tulancingo',
 'df_financiamientos_2021_tulancingo',
 'df_financiamientos_2022_tulancingo',
 'df_financiamientos_2023_tulancingo']

# 1. Cantidad

In [13]:
creditos_2019_pachuca = df_financiamientos_2019_tulancingo[['id','año','mes','modalidad','monto']]
creditos_2020_pachuca = df_financiamientos_2020_tulancingo[['id','año','mes','modalidad','monto']]
creditos_2021_pachuca = df_financiamientos_2021_tulancingo[['id','año','mes','modalidad','monto']]
creditos_2022_pachuca = df_financiamientos_2022_tulancingo[['id','año','mes','modalidad','monto']]
creditos_2023_pachuca = df_financiamientos_2023_tulancingo[['id','año','mes','modalidad','monto']]

In [14]:
total_creditos_2019_pachuca = creditos_2019_pachuca.groupby('mes').count().reset_index()
total_creditos_2019_pachuca.rename(columns={'monto': 'num_registros'}, inplace=True)
total_creditos_2019_pachuca['año'] = 2019
total_creditos_2019_pachuca = total_creditos_2019_pachuca[['año', 'mes', 'num_registros']]

total_creditos_2020_pachuca = creditos_2020_pachuca.groupby('mes').count().reset_index()
total_creditos_2020_pachuca.rename(columns={'monto': 'num_registros'}, inplace=True)
total_creditos_2020_pachuca['año'] = 2020
total_creditos_2020_pachuca = total_creditos_2020_pachuca[['año', 'mes', 'num_registros']]

total_creditos_2021_pachuca = creditos_2021_pachuca.groupby('mes').count().reset_index()
total_creditos_2021_pachuca.rename(columns={'monto': 'num_registros'}, inplace=True)
total_creditos_2021_pachuca['año'] = 2021
total_creditos_2021_pachuca = total_creditos_2021_pachuca[['año', 'mes', 'num_registros']]

total_creditos_2022_pachuca = creditos_2022_pachuca.groupby('mes').count().reset_index()
total_creditos_2022_pachuca.rename(columns={'monto': 'num_registros'}, inplace=True)
total_creditos_2022_pachuca['año'] = 2022
total_creditos_2022_pachuca = total_creditos_2022_pachuca[['año', 'mes', 'num_registros']]

total_creditos_2023_pachuca = creditos_2023_pachuca.groupby('mes').count().reset_index()
total_creditos_2023_pachuca.rename(columns={'monto': 'num_registros'}, inplace=True)
total_creditos_2023_pachuca['año'] = 2023
total_creditos_2023_pachuca = total_creditos_2023_pachuca[['año', 'mes', 'num_registros']]

total_conteo_creditos_pachuca = pd.concat([total_creditos_2019_pachuca, total_creditos_2020_pachuca, total_creditos_2021_pachuca, total_creditos_2022_pachuca, total_creditos_2023_pachuca], ignore_index=True)
total_conteo_creditos_pachuca.head()

,año,mes,num_registros
0,2019,1,16
1,2019,2,28
2,2019,3,29
3,2019,4,33
4,2019,5,40


In [15]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Preparar datos para el modelo
X = total_conteo_creditos_pachuca[['año', 'mes']]
y = total_conteo_creditos_pachuca['num_registros']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [16]:
from sklearn.metrics import r2_score
import numpy as np

# Hacer predicciones en el conjunto de prueba
y_pred = modelo.predict(X_test)
# Calcular el coeficiente de determinación (R^2)
r2 = r2_score(y_test, y_pred)
print("Coeficiente de determinación (R^2):", r2)

from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
print("Error cuadrático medio (MSE):", mse)

from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)
print("Error absoluto medio (MAE):", mae)
rmse = np.sqrt(mse)
print("Raíz del error cuadrático medio (RMSE):", rmse)

Coeficiente de determinación (R^2): 0.26300807947573557
Error cuadrático medio (MSE): 55.944851967019
Error absoluto medio (MAE): 6.068347856208675
Raíz del error cuadrático medio (RMSE): 7.479629132986408


In [17]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2023, 5),
                            'mes': range(8, 13)})

# DF con todas las combinaciones posibles de año y mes para 2024-2026
fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
                                  'mes': np.tile(range(1, 13), 3)})
nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)

# Redondear las predicciones a números enteros
predicciones_enteros = np.round(predicciones).astype(int)

# Agregamos las predicciones redondeadas al DataFrame de nuevas fechas
nuevas_fechas['num_registros'] = predicciones_enteros
nuevas_fechas.head()

total_conteo_pachuca = pd.concat([total_conteo_creditos_pachuca, nuevas_fechas], ignore_index=True)
total_conteo_pachuca

,año,mes,num_registros
0,2019,1,16
1,2019,2,28
2,2019,3,29
3,2019,4,33
4,2019,5,40
...,...,...,...
93,2026,8,-2
94,2026,9,-1
95,2026,10,0
96,2026,11,1


In [18]:
total_conteo_pachuca = total_conteo_pachuca.groupby('año')['num_registros'].sum().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
total_conteo_pachuca['modalidad'] = 'Créditos (todos)'
# Función para formatear números con separadores de coma y sin decimales
def format_number(x):
    return '{:,.0f}'.format(x)  # Formatear sin decimales
# Formatear la columna 'montos' con la función personalizada
total_conteo_pachuca['num_registros'] = total_conteo_pachuca['num_registros'].map(format_number)
total_conteo_pachuca

,año,num_registros,modalidad
0,2019,406,Créditos (todos)
1,2020,389,Créditos (todos)
2,2021,371,Créditos (todos)
3,2022,316,Créditos (todos)
4,2023,134,Créditos (todos)
5,2024,106,Créditos (todos)
6,2025,34,Créditos (todos)
7,2026,-37,Créditos (todos)


In [19]:
# Filtrar los datos donde 'modalidad' sea igual a '1'
creditos_2019_pachuca_modalidad_1 = creditos_2019_pachuca[creditos_2019_pachuca['modalidad'] == 1]
# Agrupar por mes y contar los registros
total_creditos_2019_pachuca = creditos_2019_pachuca_modalidad_1.groupby('mes').size().reset_index(name='num_registros')
total_creditos_2019_pachuca['año'] = 2019
total_creditos_2019_pachuca = total_creditos_2019_pachuca[['año', 'mes', 'num_registros']] # Reordenar las columnas

creditos_2020_pachuca_modalidad_1 = creditos_2020_pachuca[creditos_2020_pachuca['modalidad'] == 1]
total_creditos_2020_pachuca = creditos_2020_pachuca_modalidad_1.groupby('mes').size().reset_index(name='num_registros')
total_creditos_2020_pachuca['año'] = 2020
total_creditos_2020_pachuca = total_creditos_2020_pachuca[['año', 'mes', 'num_registros']]

creditos_2021_pachuca_modalidad_1 = creditos_2021_pachuca[creditos_2021_pachuca['modalidad'] == 1]
total_creditos_2021_pachuca = creditos_2021_pachuca_modalidad_1.groupby('mes').size().reset_index(name='num_registros')
total_creditos_2021_pachuca['año'] = 2021
total_creditos_2021_pachuca = total_creditos_2021_pachuca[['año', 'mes', 'num_registros']]

creditos_2022_pachuca_modalidad_1 = creditos_2022_pachuca[creditos_2022_pachuca['modalidad'] == 1]
total_creditos_2022_pachuca = creditos_2022_pachuca_modalidad_1.groupby('mes').size().reset_index(name='num_registros')
total_creditos_2022_pachuca['año'] = 2022
total_creditos_2022_pachuca = total_creditos_2022_pachuca[['año', 'mes', 'num_registros']]

creditos_2023_pachuca_modalidad_1 = creditos_2023_pachuca[creditos_2023_pachuca['modalidad'] == 1]
total_creditos_2023_pachuca = creditos_2023_pachuca_modalidad_1.groupby('mes').size().reset_index(name='num_registros')
total_creditos_2023_pachuca['año'] = 2023
total_creditos_2023_pachuca = total_creditos_2023_pachuca[['año', 'mes', 'num_registros']]

nueva_conteo_creditos_pachuca = pd.concat([total_creditos_2019_pachuca, total_creditos_2020_pachuca, total_creditos_2021_pachuca, total_creditos_2022_pachuca, total_creditos_2023_pachuca], ignore_index=True)
nueva_conteo_creditos_pachuca.head()

,año,mes,num_registros
0,2019,1,7
1,2019,2,8
2,2019,3,7
3,2019,4,10
4,2019,5,17


In [20]:
# Preparar datos para el modelo
X = nueva_conteo_creditos_pachuca[['año', 'mes']]
y = nueva_conteo_creditos_pachuca['num_registros']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [21]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2023, 5),
                            'mes': range(8, 13)})

# DF con todas las combinaciones posibles de año y mes para 2024-2026
fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
                                  'mes': np.tile(range(1, 13), 3)})
nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)
predicciones_enteros = np.round(predicciones).astype(int) # Redondear las predicciones a números enteros
# Agregamos las predicciones redondeadas al DataFrame de nuevas fechas
nuevas_fechas['num_registros'] = predicciones_enteros

nueva_conteo_pachuca = pd.concat([nueva_conteo_creditos_pachuca, nuevas_fechas], ignore_index=True)
nueva_conteo_pachuca = nueva_conteo_pachuca.groupby('año')['num_registros'].sum().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
nueva_conteo_pachuca['modalidad'] = 'Nueva'

# Función para formatear números con separadores de coma y sin decimales
def format_number(x):
    return '{:,.0f}'.format(x)  # Formatear sin decimales

# Formatear la columna 'num_registros' con la función personalizada
nueva_conteo_pachuca['num_registros'] = nueva_conteo_pachuca['num_registros'].map(format_number)
nueva_conteo_pachuca

,año,num_registros,modalidad
0,2019,133,Nueva
1,2020,98,Nueva
2,2021,80,Nueva
3,2022,95,Nueva
4,2023,78,Nueva
5,2024,59,Nueva
6,2025,46,Nueva
7,2026,34,Nueva


In [22]:
# Filtrar los datos donde 'modalidad' sea igual a '2'
creditos_2019_pachuca_modalidad_1 = creditos_2019_pachuca[creditos_2019_pachuca['modalidad'] == 2]
# Agrupar por mes y contar los registros
total_creditos_2019_pachuca = creditos_2019_pachuca_modalidad_1.groupby('mes').size().reset_index(name='num_registros')
total_creditos_2019_pachuca['año'] = 2019
total_creditos_2019_pachuca = total_creditos_2019_pachuca[['año', 'mes', 'num_registros']] # Reordenar las columnas

creditos_2020_pachuca_modalidad_1 = creditos_2020_pachuca[creditos_2020_pachuca['modalidad'] == 2]
total_creditos_2020_pachuca = creditos_2020_pachuca_modalidad_1.groupby('mes').size().reset_index(name='num_registros')
total_creditos_2020_pachuca['año'] = 2020
total_creditos_2020_pachuca = total_creditos_2020_pachuca[['año', 'mes', 'num_registros']]

creditos_2021_pachuca_modalidad_1 = creditos_2021_pachuca[creditos_2021_pachuca['modalidad'] == 2]
total_creditos_2021_pachuca = creditos_2021_pachuca_modalidad_1.groupby('mes').size().reset_index(name='num_registros')
total_creditos_2021_pachuca['año'] = 2021
total_creditos_2021_pachuca = total_creditos_2021_pachuca[['año', 'mes', 'num_registros']]

creditos_2022_pachuca_modalidad_1 = creditos_2022_pachuca[creditos_2022_pachuca['modalidad'] == 2]
total_creditos_2022_pachuca = creditos_2022_pachuca_modalidad_1.groupby('mes').size().reset_index(name='num_registros')
total_creditos_2022_pachuca['año'] = 2022
total_creditos_2022_pachuca = total_creditos_2022_pachuca[['año', 'mes', 'num_registros']]

creditos_2023_pachuca_modalidad_1 = creditos_2023_pachuca[creditos_2023_pachuca['modalidad'] == 2]
total_creditos_2023_pachuca = creditos_2023_pachuca_modalidad_1.groupby('mes').size().reset_index(name='num_registros')
total_creditos_2023_pachuca['año'] = 2023
total_creditos_2023_pachuca = total_creditos_2023_pachuca[['año', 'mes', 'num_registros']]

# Concatenar los DataFrames resultantes para cada año
mejoramiento_conteo_pachuca = pd.concat([total_creditos_2019_pachuca, total_creditos_2020_pachuca,total_creditos_2021_pachuca,total_creditos_2022_pachuca,total_creditos_2023_pachuca], ignore_index=True)
mejoramiento_conteo_pachuca.head()

,año,mes,num_registros
0,2019,1,2
1,2019,2,7
2,2019,3,7
3,2019,4,12
4,2019,5,8


In [23]:
# Preparar datos para el modelo
X = mejoramiento_conteo_pachuca[['año', 'mes']]
y = mejoramiento_conteo_pachuca['num_registros']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [24]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2023, 5),
                            'mes': range(8, 13)})

# DF con todas las combinaciones posibles de año y mes para 2024-2026
fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
                                  'mes': np.tile(range(1, 13), 3)})
nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)

# Redondear las predicciones a números enteros
predicciones_enteros = np.round(predicciones).astype(int)

# Agregamos las predicciones redondeadas al DataFrame de nuevas fechas
nuevas_fechas['num_registros'] = predicciones_enteros
mejoramiento_conteo_pachuca = pd.concat([mejoramiento_conteo_pachuca, nuevas_fechas], ignore_index=True)
mejoramiento_conteo_pachuca = mejoramiento_conteo_pachuca.groupby('año')['num_registros'].sum().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
mejoramiento_conteo_pachuca['modalidad'] = 'Mejoramientos'

# Función para formatear números con separadores de coma y sin decimales
def format_number(x):
    return '{:,.0f}'.format(x)  # Formatear sin decimales

# Formatear la columna 'num_registros' con la función personalizada
mejoramiento_conteo_pachuca['num_registros'] = mejoramiento_conteo_pachuca['num_registros'].map(format_number)
mejoramiento_conteo_pachuca

,año,num_registros,modalidad
0,2019,81,Mejoramientos
1,2020,90,Mejoramientos
2,2021,67,Mejoramientos
3,2022,41,Mejoramientos
4,2023,16,Mejoramientos
5,2024,23,Mejoramientos
6,2025,10,Mejoramientos
7,2026,-4,Mejoramientos


In [25]:
# Filtrar los datos donde 'modalidad' sea igual a '2'
creditos_2019_pachuca_modalidad_1 = creditos_2019_pachuca[creditos_2019_pachuca['modalidad'] == 3]
# Agrupar por mes y contar los registros
total_creditos_2019_pachuca = creditos_2019_pachuca_modalidad_1.groupby('mes').size().reset_index(name='num_registros')
total_creditos_2019_pachuca['año'] = 2019
total_creditos_2019_pachuca = total_creditos_2019_pachuca[['año', 'mes', 'num_registros']] # Reordenar las columnas

creditos_2020_pachuca_modalidad_1 = creditos_2020_pachuca[creditos_2020_pachuca['modalidad'] == 3]
total_creditos_2020_pachuca = creditos_2020_pachuca_modalidad_1.groupby('mes').size().reset_index(name='num_registros')
total_creditos_2020_pachuca['año'] = 2020
total_creditos_2020_pachuca = total_creditos_2020_pachuca[['año', 'mes', 'num_registros']]

creditos_2021_pachuca_modalidad_1 = creditos_2021_pachuca[creditos_2021_pachuca['modalidad'] == 3]
total_creditos_2021_pachuca = creditos_2021_pachuca_modalidad_1.groupby('mes').size().reset_index(name='num_registros')
total_creditos_2021_pachuca['año'] = 2021
total_creditos_2021_pachuca = total_creditos_2021_pachuca[['año', 'mes', 'num_registros']]

creditos_2022_pachuca_modalidad_1 = creditos_2022_pachuca[creditos_2022_pachuca['modalidad'] == 3]
total_creditos_2022_pachuca = creditos_2022_pachuca_modalidad_1.groupby('mes').size().reset_index(name='num_registros')
total_creditos_2022_pachuca['año'] = 2022
total_creditos_2022_pachuca = total_creditos_2022_pachuca[['año', 'mes', 'num_registros']]

creditos_2023_pachuca_modalidad_1 = creditos_2023_pachuca[creditos_2023_pachuca['modalidad'] == 3]
total_creditos_2023_pachuca = creditos_2023_pachuca_modalidad_1.groupby('mes').size().reset_index(name='num_registros')
total_creditos_2023_pachuca['año'] = 2023
total_creditos_2023_pachuca = total_creditos_2023_pachuca[['año', 'mes', 'num_registros']]

# Concatenar los DataFrames resultantes para cada año
usada_conteo_pachuca = pd.concat([total_creditos_2019_pachuca, total_creditos_2020_pachuca,total_creditos_2021_pachuca,total_creditos_2022_pachuca,total_creditos_2023_pachuca], ignore_index=True)
usada_conteo_pachuca.head()

,año,mes,num_registros
0,2019,1,7
1,2019,2,13
2,2019,3,15
3,2019,4,11
4,2019,5,15


In [26]:
# Preparar datos para el modelo
X = usada_conteo_pachuca[['año', 'mes']]
y = usada_conteo_pachuca['num_registros']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [27]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2023, 5),
                            'mes': range(8, 13)})

# DF con todas las combinaciones posibles de año y mes para 2024-2026
fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
                                  'mes': np.tile(range(1, 13), 3)})
nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)

# Redondear las predicciones a números enteros
predicciones_enteros = np.round(predicciones).astype(int)

# Agregamos las predicciones redondeadas al DataFrame de nuevas fechas
nuevas_fechas['num_registros'] = predicciones_enteros
usada_conteo_pachuca = pd.concat([usada_conteo_pachuca, nuevas_fechas], ignore_index=True)
usada_conteo_pachuca = usada_conteo_pachuca.groupby('año')['num_registros'].sum().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
usada_conteo_pachuca['modalidad'] = 'Usada'

# Función para formatear números con separadores de coma y sin decimales
def format_number(x):
    return '{:,.0f}'.format(x)  # Formatear sin decimales

# Formatear la columna 'num_registros' con la función personalizada
usada_conteo_pachuca['num_registros'] = usada_conteo_pachuca['num_registros'].map(format_number)
usada_conteo_pachuca

,año,num_registros,modalidad
0,2019,190,Usada
1,2020,195,Usada
2,2021,214,Usada
3,2022,161,Usada
4,2023,85,Usada
5,2024,158,Usada
6,2025,151,Usada
7,2026,143,Usada


In [28]:
# Filtrar los datos donde 'modalidad' sea igual a '2'
creditos_2019_pachuca_modalidad_1 = creditos_2019_pachuca[creditos_2019_pachuca['modalidad'] == 4]
# Agrupar por mes y contar los registros
total_creditos_2019_pachuca = creditos_2019_pachuca_modalidad_1.groupby('mes').size().reset_index(name='num_registros')
total_creditos_2019_pachuca['año'] = 2019
total_creditos_2019_pachuca = total_creditos_2019_pachuca[['año', 'mes', 'num_registros']] # Reordenar las columnas

creditos_2020_pachuca_modalidad_1 = creditos_2020_pachuca[creditos_2020_pachuca['modalidad'] == 4]
total_creditos_2020_pachuca = creditos_2020_pachuca_modalidad_1.groupby('mes').size().reset_index(name='num_registros')
total_creditos_2020_pachuca['año'] = 2020
total_creditos_2020_pachuca = total_creditos_2020_pachuca[['año', 'mes', 'num_registros']]

creditos_2021_pachuca_modalidad_1 = creditos_2021_pachuca[creditos_2021_pachuca['modalidad'] == 4]
total_creditos_2021_pachuca = creditos_2021_pachuca_modalidad_1.groupby('mes').size().reset_index(name='num_registros')
total_creditos_2021_pachuca['año'] = 2021
total_creditos_2021_pachuca = total_creditos_2021_pachuca[['año', 'mes', 'num_registros']]

creditos_2022_pachuca_modalidad_1 = creditos_2022_pachuca[creditos_2022_pachuca['modalidad'] == 4]
total_creditos_2022_pachuca = creditos_2022_pachuca_modalidad_1.groupby('mes').size().reset_index(name='num_registros')
total_creditos_2022_pachuca['año'] = 2022
total_creditos_2022_pachuca = total_creditos_2022_pachuca[['año', 'mes', 'num_registros']]

creditos_2023_pachuca_modalidad_1 = creditos_2023_pachuca[creditos_2023_pachuca['modalidad'] == 4]
total_creditos_2023_pachuca = creditos_2023_pachuca_modalidad_1.groupby('mes').size().reset_index(name='num_registros')
total_creditos_2023_pachuca['año'] = 2023
total_creditos_2023_pachuca = total_creditos_2023_pachuca[['año', 'mes', 'num_registros']]

# Concatenar los DataFrames resultantes para cada año
otros_conteo_pachuca = pd.concat([total_creditos_2019_pachuca, total_creditos_2020_pachuca,total_creditos_2021_pachuca,total_creditos_2022_pachuca,total_creditos_2023_pachuca], ignore_index=True)
otros_conteo_pachuca.head()

,año,mes,num_registros
0,2019,7,1
1,2019,9,1
2,2020,2,1
3,2020,5,1
4,2020,7,1


In [29]:
# Preparar datos para el modelo
X = otros_conteo_pachuca[['año', 'mes']]
y = otros_conteo_pachuca['num_registros']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [30]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2023, 5),
                            'mes': range(8, 13)})

# DF con todas las combinaciones posibles de año y mes para 2024-2026
fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
                                  'mes': np.tile(range(1, 13), 3)})
nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)

# Redondear las predicciones a números enteros
predicciones_enteros = np.round(predicciones).astype(int)

# Agregamos las predicciones redondeadas al DataFrame de nuevas fechas
nuevas_fechas['num_registros'] = predicciones_enteros
otros_conteo_pachuca = pd.concat([otros_conteo_pachuca, nuevas_fechas], ignore_index=True)
otros_conteo_pachuca = otros_conteo_pachuca.groupby('año')['num_registros'].sum().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
otros_conteo_pachuca['modalidad'] = 'Otros'

# Función para formatear números con separadores de coma y sin decimales
def format_number(x):
    return '{:,.0f}'.format(x)  # Formatear sin decimales

# Formatear la columna 'num_registros' con la función personalizada
otros_conteo_pachuca['num_registros'] = otros_conteo_pachuca['num_registros'].map(format_number)
otros_conteo_pachuca

,año,num_registros,modalidad
0,2019,2,Otros
1,2020,6,Otros
2,2021,10,Otros
3,2022,19,Otros
4,2023,15,Otros
5,2024,36,Otros
6,2025,48,Otros
7,2026,48,Otros


In [32]:
conteo_pachuca = pd.concat([
    total_conteo_pachuca,
    nueva_conteo_pachuca,
    mejoramiento_conteo_pachuca,
    usada_conteo_pachuca,
    otros_conteo_pachuca,
])
# Datos
año = conteo_pachuca['año']
modalidades = conteo_pachuca['modalidad']
creditos = conteo_pachuca['num_registros']

fig = go.Figure()
# Agregar líneas para cada modalidad
for modalidad in modalidades.unique():
    df_modalidad = conteo_pachuca[conteo_pachuca['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['año'],
        y=df_modalidad['num_registros'].apply(lambda x: float(x.replace(',', ''))),  # Usar los montos limpios
        mode='markers+lines',
        name=modalidad
    ))
fig.update_layout(
    title='Créditos',
    yaxis=dict(gridcolor='#dddcda'),
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

# Agregar etiquetas de texto a los puntos
#for trace in fig.data:
#    df_modalidad = conteo_pachuca[conteo_pachuca['modalidad'] == trace.name]
#    for i, point in enumerate(trace.y):
#        fig.add_annotation(
#            x=trace.x[i], 
#            y=point, 
#            text=f'{df_modalidad.iloc[i]["num_registros"]}', 
#            showarrow=False,
#            font=dict(color='black', size=10),
#            yshift=10
#        )


# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, os.path.join(carpeta, f'{nombre_archivo}.html'))

# Exportar
guardar_grafico_como_html(fig, 'g_scatt_forecast_creditgral_cantidad', carpeta='assets/graficas')
fig.show()

In [33]:
conteo_tres_pachuca = pd.concat([
    total_conteo_pachuca,
    nueva_conteo_pachuca,
    usada_conteo_pachuca
])
# Datos
año = conteo_tres_pachuca['año']
modalidades = conteo_tres_pachuca['modalidad']
creditos = conteo_tres_pachuca['num_registros']

fig = go.Figure()
# Agregar líneas para cada modalidad
for modalidad in modalidades.unique():
    df_modalidad = conteo_tres_pachuca[conteo_tres_pachuca['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['año'],
        y=df_modalidad['num_registros'].apply(lambda x: float(x.replace(',', ''))),  # Usar los montos limpios
        mode='markers+lines',
        name=modalidad
    ))
fig.update_layout(
    title='Créditos',
    yaxis=dict(gridcolor='#dddcda'),
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)
for trace in fig.data:
    df_modalidad = conteo_tres_pachuca[conteo_tres_pachuca['modalidad'] == trace.name]
    for i, point in enumerate(trace.y):
        # No formatear el valor del monto como moneda
        fig.add_annotation(
            x=trace.x[i], 
            y=point, 
            text=f'{point}',  # Mostrar el valor simple del punto
            showarrow=False,
            font=dict(color='black', size=10),
            yshift=10
        )

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, os.path.join(carpeta, f'{nombre_archivo}.html'))

# Exportar
guardar_grafico_como_html(fig, 'g_scatt_forecast_creditgral_cantidad', carpeta='assets/graficas')
fig.show()

## *DIRECTOS*

In [34]:
data_3 = {
    'Año': [2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026],
    'Créditos (todos)': [406,	389,	371,	317,	222,	259,	304,	355],
    'Vivienda Nueva': [81,	90,	67,	41,	28,	31,	34,	38],
    'Vivienda Usada': [73,	94,	89,	91,	45,	58,	59,	60],
}
df_3 = pd.DataFrame(data_3)

# Reemplazar los símbolos y convertir a float
#for col in df_3.columns:
 #   if col != 'Año':
  #      df_3[col] = df_3[col].str.replace('$', '').str.replace(',', '').astype(float)

# Reestructurar los datos
trimestres = []
modalidades = []
montos = []

for index, row in df_3.iterrows():
    for col in df_3.columns[1:]:  # Excluimos la columna de Año
        trimestres.append(row['Año'])
        modalidades.append(col)
        montos.append(row[col])

# Crear el nuevo DataFrame
nuevo_df_3 = pd.DataFrame({
    'trimestres': trimestres,
    'modalidades': modalidades,
    'montos': montos
})
nuevo_df_3

,trimestres,modalidades,montos
0,2019,Créditos (todos),406
1,2019,Vivienda Nueva,81
2,2019,Vivienda Usada,73
3,2020,Créditos (todos),389
4,2020,Vivienda Nueva,90
5,2020,Vivienda Usada,94
6,2021,Créditos (todos),371
7,2021,Vivienda Nueva,67
8,2021,Vivienda Usada,89
9,2022,Créditos (todos),317


In [35]:
import plotly.graph_objects as go
import os
import plotly.io as pio
import locale

# Datos
trimestres = nuevo_df_3['trimestres']
modalidades = nuevo_df_3['modalidades']
montos = nuevo_df_3['montos']

fig = go.Figure()

# Agregar líneas para cada modalidad
for modalidad in modalidades.unique():
    df_modalidad = nuevo_df_3[nuevo_df_3['modalidades'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['trimestres'],
        y=df_modalidad['montos'],
        mode='markers+lines',
        name=modalidad
    ))

fig.update_layout(
    title='Créditos',
    yaxis=dict(gridcolor='#dddcda'),
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

for trace in fig.data:
    df_modalidad = nuevo_df_3[nuevo_df_3['modalidades'] == trace.name]
    for i, point in enumerate(trace.y):
        # No formatear el valor del monto como moneda
        fig.add_annotation(
            x=trace.x[i], 
            y=point, 
            text=f'{point}',  # Mostrar el valor simple del punto
            showarrow=False,
            font=dict(color='black', size=10),
            yshift=10
        )

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, os.path.join(carpeta, f'{nombre_archivo}.html'))

# Exportar
guardar_grafico_como_html(fig, 'g_scatt_forecast_creditgral_cantidad_direct', carpeta='assets/graficas')
fig.show()

# 2. Total

### Base

In [38]:
creditos_2019_pachuca = df_financiamientos_2019_tulancingo[['id','año','mes','modalidad','monto']]
creditos_2020_pachuca = df_financiamientos_2020_tulancingo[['id','año','mes','modalidad','monto']]
creditos_2021_pachuca = df_financiamientos_2021_tulancingo[['id','año','mes','modalidad','monto']]
creditos_2022_pachuca = df_financiamientos_2022_tulancingo[['id','año','mes','modalidad','monto']]
creditos_2023_pachuca = df_financiamientos_2023_tulancingo[['id','año','mes','modalidad','monto']]

In [39]:
total_creditos_2019_pachuca = creditos_2019_pachuca.groupby('mes').sum().reset_index()
total_creditos_2019_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2019_pachuca['año'] = 2019
total_creditos_2019_pachuca = total_creditos_2019_pachuca[['año', 'mes', 'montos']]

total_creditos_2020_pachuca = creditos_2020_pachuca.groupby('mes').sum().reset_index()
total_creditos_2020_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2020_pachuca['año'] = 2020
total_creditos_2020_pachuca = total_creditos_2020_pachuca[['año', 'mes', 'montos']]

total_creditos_2021_pachuca = creditos_2021_pachuca.groupby('mes').sum().reset_index()
total_creditos_2021_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2021_pachuca['año'] = 2021
total_creditos_2021_pachuca = total_creditos_2021_pachuca[['año', 'mes', 'montos']]

total_creditos_2022_pachuca = creditos_2022_pachuca.groupby('mes').sum().reset_index()
total_creditos_2022_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2022_pachuca['año'] = 2022
total_creditos_2022_pachuca = total_creditos_2022_pachuca[['año', 'mes', 'montos']]

total_creditos_2023_pachuca = creditos_2023_pachuca.groupby('mes').sum().reset_index()
total_creditos_2023_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2023_pachuca['año'] = 2023
total_creditos_2023_pachuca = total_creditos_2023_pachuca[['año', 'mes', 'montos']]

total_conteo_creditos_pachuca = pd.concat([total_creditos_2019_pachuca, total_creditos_2020_pachuca, total_creditos_2021_pachuca, total_creditos_2022_pachuca, total_creditos_2023_pachuca], ignore_index=True)
total_conteo_creditos_pachuca.head()

,año,mes,montos
0,2019,1,11787572.19
1,2019,2,16520649.45
2,2019,3,14752147.62
3,2019,4,14434520.60
4,2019,5,27661094.33


In [40]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np

# Preparar datos para el modelo
X = total_conteo_creditos_pachuca[['año', 'mes']]
y = total_conteo_creditos_pachuca['montos']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [41]:
from sklearn.metrics import r2_score
# Hacer predicciones en el conjunto de prueba
y_pred = modelo.predict(X_test)
# Calcular el coeficiente de determinación (R^2)
r2 = r2_score(y_test, y_pred)
print("Coeficiente de determinación (R^2):", r2)

from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
print("Error cuadrático medio (MSE):", mse)

from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)
print("Error absoluto medio (MAE):", mae)
rmse = np.sqrt(mse)
print("Raíz del error cuadrático medio (RMSE):", rmse)

Coeficiente de determinación (R^2): 0.07966522575338475
Error cuadrático medio (MSE): 63093990763651.055
Error absoluto medio (MAE): 6991538.043648444
Raíz del error cuadrático medio (RMSE): 7943172.588056428


In [42]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2023, 5),
                            'mes': range(8, 13)})

# DF con todas las combinaciones posibles de año y mes para 2024-2026
fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
                                  'mes': np.tile(range(1, 13), 3)})
nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)
# Agregamos las predicciones al DataFrame de nuevas fechas
nuevas_fechas['montos'] = predicciones
total_pachuca = pd.concat([total_conteo_creditos_pachuca, nuevas_fechas], ignore_index=True)
total_todos_pachuca = total_pachuca.groupby('año')['montos'].sum().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
total_todos_pachuca['modalidad'] = 'Créditos (todos)'
# Función para formatear números con separadores de coma y dos decimales
def format_number(x):
   return '{:,.2f}'.format(x)
# Formatear la columna 'montos' con la función personalizada
total_todos_pachuca['montos'] = total_todos_pachuca['montos'].map(format_number)
total_todos_pachuca

,año,montos,modalidad
0,2019,"259,936,080.61",Créditos (todos)
1,2020,"267,516,703.55",Créditos (todos)
2,2021,"289,356,367.89",Créditos (todos)
3,2022,"264,352,549.14",Créditos (todos)
4,2023,"135,366,517.01",Créditos (todos)
5,2024,"133,392,907.29",Créditos (todos)
6,2025,"98,402,819.44",Créditos (todos)
7,2026,"63,412,731.59",Créditos (todos)


In [43]:
# Filtrar los datos donde 'modalidad' sea igual a '1'
creditos_2019_pachuca_modalidad_1 = creditos_2019_pachuca[creditos_2019_pachuca['modalidad'] == 1]
# Agrupar por mes y sumar los montos
total_creditos_2019_pachuca = creditos_2019_pachuca_modalidad_1.groupby('mes')['monto'].sum().reset_index()
total_creditos_2019_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2019_pachuca['año'] = 2019
total_creditos_2019_pachuca = total_creditos_2019_pachuca[['año', 'mes', 'montos']] # Reordenar las columnas

# Filtrar los datos donde 'modalidad' sea igual a '1'
creditos_2020_pachuca_modalidad_1 = creditos_2020_pachuca[creditos_2020_pachuca['modalidad'] == 1]
# Agrupar por mes y sumar los montos
total_creditos_2020_pachuca = creditos_2020_pachuca_modalidad_1.groupby('mes')['monto'].sum().reset_index()
total_creditos_2020_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2020_pachuca['año'] = 2020
total_creditos_2020_pachuca = total_creditos_2020_pachuca[['año', 'mes', 'montos']] # Reordenar las columnas

# Filtrar los datos donde 'modalidad' sea igual a '1'
creditos_2021_pachuca_modalidad_1 = creditos_2021_pachuca[creditos_2021_pachuca['modalidad'] == 1]
# Agrupar por mes y sumar los montos
total_creditos_2021_pachuca = creditos_2021_pachuca_modalidad_1.groupby('mes')['monto'].sum().reset_index()
total_creditos_2021_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2021_pachuca['año'] = 2021
total_creditos_2021_pachuca = total_creditos_2021_pachuca[['año', 'mes', 'montos']] # Reordenar las columnas

# Filtrar los datos donde 'modalidad' sea igual a '1'
creditos_2022_pachuca_modalidad_1 = creditos_2022_pachuca[creditos_2022_pachuca['modalidad'] == 1]
# Agrupar por mes y sumar los montos
total_creditos_2022_pachuca = creditos_2022_pachuca_modalidad_1.groupby('mes')['monto'].sum().reset_index()
total_creditos_2022_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2022_pachuca['año'] = 2022
total_creditos_2022_pachuca = total_creditos_2022_pachuca[['año', 'mes', 'montos']] # Reordenar las columnas

# Filtrar los datos donde 'modalidad' sea igual a '1'
creditos_2023_pachuca_modalidad_1 = creditos_2023_pachuca[creditos_2023_pachuca['modalidad'] == 1]
# Agrupar por mes y sumar los montos
total_creditos_2023_pachuca = creditos_2023_pachuca_modalidad_1.groupby('mes')['monto'].sum().reset_index()
total_creditos_2023_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2023_pachuca['año'] = 2023
total_creditos_2023_pachuca = total_creditos_2023_pachuca[['año', 'mes', 'montos']] # Reordenar las columnas

total_creditos_nueva_pachuca = pd.concat([total_creditos_2019_pachuca, total_creditos_2020_pachuca, total_creditos_2021_pachuca, total_creditos_2022_pachuca, total_creditos_2023_pachuca], ignore_index=True)
total_creditos_nueva_pachuca.head()

,año,mes,montos
0,2019,1,6823749.78
1,2019,2,6745626.45
2,2019,3,4913661.59
3,2019,4,7475032.19
4,2019,5,16318059.94


In [44]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np

# Preparar datos para el modelo
X = total_creditos_nueva_pachuca[['año', 'mes']]
y = total_creditos_nueva_pachuca['montos']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [45]:
from sklearn.metrics import r2_score
# Hacer predicciones en el conjunto de prueba
y_pred = modelo.predict(X_test)
# Calcular el coeficiente de determinación (R^2)
r2 = r2_score(y_test, y_pred)
print("Coeficiente de determinación (R^2):", r2)

from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
print("Error cuadrático medio (MSE):", mse)

from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)
print("Error absoluto medio (MAE):", mae)
rmse = np.sqrt(mse)
print("Raíz del error cuadrático medio (RMSE):", rmse)

Coeficiente de determinación (R^2): 0.004837990337749121
Error cuadrático medio (MSE): 14728987511066.486
Error absoluto medio (MAE): 3021973.4083438586
Raíz del error cuadrático medio (RMSE): 3837836.305923754


In [46]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2023, 5),
                            'mes': range(8, 13)})

# DF con todas las combinaciones posibles de año y mes para 2024-2026
fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
                                  'mes': np.tile(range(1, 13), 3)})
nuevas_fechas_nueva = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas_nueva)
# Agregamos las predicciones al DataFrame de nuevas fechas
nuevas_fechas_nueva['montos'] = predicciones
total_nueva_pachuca = pd.concat([total_creditos_nueva_pachuca, nuevas_fechas_nueva], ignore_index=True)
total_nueva_pachuca = total_nueva_pachuca.groupby('año')['montos'].sum().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
total_nueva_pachuca['modalidad'] = 'Nueva'
# Función para formatear números con separadores de coma y dos decimales
def format_number(x):
   return '{:,.2f}'.format(x)
total_nueva_pachuca['montos'] = total_nueva_pachuca['montos'].map(format_number)
total_nueva_pachuca

,año,montos,modalidad
0,2019,"110,576,187.62",Nueva
1,2020,"108,946,882.44",Nueva
2,2021,"83,552,133.83",Nueva
3,2022,"93,455,838.96",Nueva
4,2023,"88,973,049.15",Nueva
5,2024,"76,391,022.27",Nueva
6,2025,"70,163,752.07",Nueva
7,2026,"63,936,481.87",Nueva


In [47]:
# Filtrar los datos donde 'modalidad' sea igual a '3'
creditos_2019_pachuca_modalidad_3 = creditos_2019_pachuca[creditos_2019_pachuca['modalidad'] == 3]
# Agrupar por mes y sumar los montos
total_creditos_2019_pachuca = creditos_2019_pachuca_modalidad_3.groupby('mes')['monto'].sum().reset_index()
total_creditos_2019_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2019_pachuca['año'] = 2019
total_creditos_2019_pachuca = total_creditos_2019_pachuca[['año', 'mes', 'montos']] # Reordenar las columnas

# Filtrar los datos donde 'modalidad' sea igual a '3'
creditos_2020_pachuca_modalidad_3 = creditos_2020_pachuca[creditos_2020_pachuca['modalidad'] == 3]
# Agrupar por mes y sumar los montos
total_creditos_2020_pachuca = creditos_2020_pachuca_modalidad_3.groupby('mes')['monto'].sum().reset_index()
total_creditos_2020_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2020_pachuca['año'] = 2020
total_creditos_2020_pachuca = total_creditos_2020_pachuca[['año', 'mes', 'montos']] # Reordenar las columnas

creditos_2021_pachuca_modalidad_3 = creditos_2021_pachuca[creditos_2021_pachuca['modalidad'] == 3]
# Agrupar por mes y sumar los montos
total_creditos_2021_pachuca = creditos_2021_pachuca_modalidad_3.groupby('mes')['monto'].sum().reset_index()
total_creditos_2021_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2021_pachuca['año'] = 2021
total_creditos_2021_pachuca = total_creditos_2021_pachuca[['año', 'mes', 'montos']] # Reordenar las columnas

creditos_2022_pachuca_modalidad_3 = creditos_2022_pachuca[creditos_2022_pachuca['modalidad'] == 3]
# Agrupar por mes y sumar los montos
total_creditos_2022_pachuca = creditos_2022_pachuca_modalidad_3.groupby('mes')['monto'].sum().reset_index()
total_creditos_2022_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2022_pachuca['año'] = 2022
total_creditos_2022_pachuca = total_creditos_2022_pachuca[['año', 'mes', 'montos']] # Reordenar las columnas

creditos_2023_pachuca_modalidad_3 = creditos_2023_pachuca[creditos_2023_pachuca['modalidad'] == 3]
# Agrupar por mes y sumar los montos
total_creditos_2023_pachuca = creditos_2023_pachuca_modalidad_3.groupby('mes')['monto'].sum().reset_index()
total_creditos_2023_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2023_pachuca['año'] = 2023
total_creditos_2023_pachuca = total_creditos_2023_pachuca[['año', 'mes', 'montos']] # Reordenar las columnas

total_creditos_usada_pachuca = pd.concat([total_creditos_2019_pachuca, total_creditos_2020_pachuca, total_creditos_2021_pachuca, total_creditos_2022_pachuca, total_creditos_2023_pachuca], ignore_index=True)
total_creditos_usada_pachuca.head()

,año,mes,montos
0,2019,1,4931701.41
1,2019,2,9680889.00
2,2019,3,9788315.03
3,2019,4,6817423.41
4,2019,5,11201768.39


In [48]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np
# Preparar datos para el modelo
X = total_creditos_usada_pachuca[['año', 'mes']]
y = total_creditos_usada_pachuca['montos']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [49]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2023, 5),
                            'mes': range(8, 13)})

# DF con todas las combinaciones posibles de año y mes para 2024-2026
fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
                                  'mes': np.tile(range(1, 13), 3)})
nuevas_fechas_usada = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas_usada)
# Agregamos las predicciones al DataFrame de nuevas fechas
nuevas_fechas_usada['montos'] = predicciones
total_usada_pachuca = pd.concat([total_creditos_usada_pachuca, nuevas_fechas_usada], ignore_index=True)
total_usada_pachuca = total_usada_pachuca.groupby('año')['montos'].sum().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
total_usada_pachuca['modalidad'] = 'Usada'
# Función para formatear números con separadores de coma y dos decimales
def format_number(x):
   return '{:,.2f}'.format(x)
total_usada_pachuca['montos'] = total_usada_pachuca['montos'].map(format_number)
total_usada_pachuca

,año,montos,modalidad
0,2019,"144,769,506.80",Usada
1,2020,"151,737,369.43",Usada
2,2021,"182,709,207.64",Usada
3,2022,"156,176,801.18",Usada
4,2023,"86,914,291.29",Usada
5,2024,"177,561,218.24",Usada
6,2025,"183,563,358.55",Usada
7,2026,"189,565,498.87",Usada


In [50]:
forecast_creditosgral_totales_pachuca = pd.concat([total_todos_pachuca, total_nueva_pachuca,total_usada_pachuca], ignore_index=True)
# Datos
trimestres = forecast_creditosgral_totales_pachuca['año']
modalidades = forecast_creditosgral_totales_pachuca['modalidad']
montos = forecast_creditosgral_totales_pachuca['montos']

# Limpiar las cadenas de montos y convertirlas a números
promedios_limpios = [float(montos.replace(',', '')) for montos in montos]

fig = go.Figure()
# Agregar líneas para cada modalidad
for modalidad in modalidades.unique():
    df_modalidad = forecast_creditosgral_totales_pachuca[forecast_creditosgral_totales_pachuca['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['año'],
        y=df_modalidad['montos'].apply(lambda x: float(x.replace(',', ''))),  # Usar los montos limpios
        mode='markers+lines',
        name=modalidad
    ))

fig.update_layout(
    title='Monto total anual',
    yaxis=dict(gridcolor='#dddcda'),
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

# Agregar etiquetas de texto a los puntos
for trace in fig.data:
    df_modalidad = forecast_creditosgral_totales_pachuca[forecast_creditosgral_totales_pachuca['modalidad'] == trace.name]
    for i, point in enumerate(trace.y):
        fig.add_annotation(
            x=trace.x[i], 
            y=point, 
            text=f'{df_modalidad.iloc[i]["montos"]}', 
            showarrow=False,
            font=dict(color='black', size=10),
            yshift=10
        )

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, os.path.join(carpeta, f'{nombre_archivo}.html'))

# Exportar
guardar_grafico_como_html(fig, 'g_scatt_forecast_creditgral_total', carpeta='assets/graficas')
fig.show()

### *DIRECTOS*

In [36]:
data = {
    'Año': [2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026],
    'Créditos (todos)': ['$259,936,080.61', '$267,516,703.55', '$289,356,367.89', '$264,352,549.14', '$264,352,549.14', '$264,352,549.14', '$264,352,549.14', '$264,352,549.14'],
    'Vivienda Nueva': ['$110,576,187.62', '$108,946,882.44', '$83,552,133.83', '$93,455,838.96', '$93,455,838.96', '$93,455,838.96', '$93,455,838.96', '$93,455,838.96'],
    'Vivienda Usada': ['$144,769,506.80', '$151,737,369.43', '$182,709,207.64', '$156,176,801.18', '$156,176,801.18', '$156,176,801.18', '$156,176,801.18', '$156,176,801.18']
}
df = pd.DataFrame(data)

# Reemplazar los símbolos y convertir a float
for col in df.columns:
    if col != 'Año':
        df[col] = df[col].str.replace('$', '').str.replace(',', '').astype(float)

# Reestructurar los datos
trimestres = []
modalidades = []
montos = []

for index, row in df.iterrows():
    for col in df.columns[1:]:  # Excluimos la columna de Año
        trimestres.append(row['Año'])
        modalidades.append(col)
        montos.append(row[col])

# Crear el nuevo DataFrame
nuevo_df = pd.DataFrame({
    'trimestres': trimestres,
    'modalidades': modalidades,
    'montos': montos
})

In [37]:
import plotly.graph_objects as go
import os
import plotly.io as pio
import locale

# Configurar el locale para formatear los números como moneda
locale.setlocale(locale.LC_ALL, '')

# Datos
trimestres = nuevo_df['trimestres']
modalidades = nuevo_df['modalidades']
montos = nuevo_df['montos']

fig = go.Figure()

# Agregar líneas para cada modalidad
for modalidad in modalidades.unique():
    df_modalidad = nuevo_df[nuevo_df['modalidades'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['trimestres'],
        y=df_modalidad['montos'],
        mode='markers+lines',
        name=modalidad
    ))

fig.update_layout(
    title='Monto total anual',
    yaxis=dict(gridcolor='#dddcda'),
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

# Agregar etiquetas de texto a los puntos
for trace in fig.data:
    df_modalidad = nuevo_df[nuevo_df['modalidades'] == trace.name]
    for i, point in enumerate(trace.y):
        # Formatear el valor del monto como una cadena en formato de precio
        monto_formateado = locale.currency(df_modalidad.iloc[i]["montos"], grouping=True)
        fig.add_annotation(
            x=trace.x[i], 
            y=point, 
            text=f'{monto_formateado}', 
            showarrow=False,
            font=dict(color='black', size=10),
            yshift=10
        )


# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, os.path.join(carpeta, f'{nombre_archivo}.html'))

# Exportar
guardar_grafico_como_html(fig, 'g_scatt_forecast_creditgral_total_direct', carpeta='assets/graficas')
fig.show()


# 3. Promedio

### BASE

In [51]:
creditos_2019_pachuca = df_financiamientos_2019_tulancingo[['id','año','mes','modalidad','monto']]
creditos_2020_pachuca = df_financiamientos_2020_tulancingo[['id','año','mes','modalidad','monto']]
creditos_2021_pachuca = df_financiamientos_2021_tulancingo[['id','año','mes','modalidad','monto']]
creditos_2022_pachuca = df_financiamientos_2022_tulancingo[['id','año','mes','modalidad','monto']]
creditos_2023_pachuca = df_financiamientos_2023_tulancingo[['id','año','mes','modalidad','monto']]

In [52]:
total_creditos_2019_pachuca = creditos_2019_pachuca.groupby('mes').mean().reset_index()
total_creditos_2019_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2019_pachuca['año'] = 2019
total_creditos_2019_pachuca = total_creditos_2019_pachuca[['año', 'mes', 'montos']]

total_creditos_2020_pachuca = creditos_2020_pachuca.groupby('mes').mean().reset_index()
total_creditos_2020_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2020_pachuca['año'] = 2020
total_creditos_2020_pachuca = total_creditos_2020_pachuca[['año', 'mes', 'montos']]

total_creditos_2021_pachuca = creditos_2021_pachuca.groupby('mes').mean().reset_index()
total_creditos_2021_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2021_pachuca['año'] = 2021
total_creditos_2021_pachuca = total_creditos_2021_pachuca[['año', 'mes', 'montos']]

total_creditos_2022_pachuca = creditos_2022_pachuca.groupby('mes').mean().reset_index()
total_creditos_2022_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2022_pachuca['año'] = 2022
total_creditos_2022_pachuca = total_creditos_2022_pachuca[['año', 'mes', 'montos']]

total_creditos_2023_pachuca = creditos_2023_pachuca.groupby('mes').mean().reset_index()
total_creditos_2023_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2023_pachuca['año'] = 2023
total_creditos_2023_pachuca = total_creditos_2023_pachuca[['año', 'mes', 'montos']]

total_conteo_creditos_pachuca = pd.concat([total_creditos_2019_pachuca, total_creditos_2020_pachuca, total_creditos_2021_pachuca, total_creditos_2022_pachuca, total_creditos_2023_pachuca], ignore_index=True)
total_conteo_creditos_pachuca.head()

,año,mes,montos
0,2019,1,736723.261875
1,2019,2,590023.194643
2,2019,3,508694.745517
3,2019,4,437409.715152
4,2019,5,691527.358250


In [53]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np

# Preparar datos para el modelo
X = total_conteo_creditos_pachuca[['año', 'mes']]
y = total_conteo_creditos_pachuca['montos']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [54]:
from sklearn.metrics import r2_score
# Hacer predicciones en el conjunto de prueba
y_pred = modelo.predict(X_test)
# Calcular el coeficiente de determinación (R^2)
r2 = r2_score(y_test, y_pred)
print("Coeficiente de determinación (R^2):", r2)

from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
print("Error cuadrático medio (MSE):", mse)

from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)
print("Error absoluto medio (MAE):", mae)
rmse = np.sqrt(mse)
print("Raíz del error cuadrático medio (RMSE):", rmse)

Coeficiente de determinación (R^2): 0.47572350438075084
Error cuadrático medio (MSE): 23508246557.027332
Error absoluto medio (MAE): 124637.96642722591
Raíz del error cuadrático medio (RMSE): 153323.99211156528


In [55]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2023, 5),
                            'mes': range(8, 13)})

# DF con todas las combinaciones posibles de año y mes para 2024-2026
fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
                                  'mes': np.tile(range(1, 13), 3)})
nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)
# Agregamos las predicciones al DataFrame de nuevas fechas
nuevas_fechas['montos'] = predicciones
total_pachuca = pd.concat([total_conteo_creditos_pachuca, nuevas_fechas], ignore_index=True)
total_todos_pachuca = total_pachuca.groupby('año')['montos'].sum().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
total_todos_pachuca['modalidad'] = 'Créditos (todos)'
# Función para formatear números con separadores de coma y dos decimales
def format_number(x):
   return '{:,.2f}'.format(x)
# Formatear la columna 'montos' con la función personalizada
total_todos_pachuca['montos'] = total_todos_pachuca['montos'].map(format_number)
total_todos_pachuca

,año,montos,modalidad
0,2019,"7,659,394.48",Créditos (todos)
1,2020,"8,087,510.13",Créditos (todos)
2,2021,"9,267,657.80",Créditos (todos)
3,2022,"10,007,968.76",Créditos (todos)
4,2023,"14,693,450.49",Créditos (todos)
5,2024,"13,106,683.63",Créditos (todos)
6,2025,"14,284,632.47",Créditos (todos)
7,2026,"15,462,581.32",Créditos (todos)


In [56]:
# Filtrar los datos donde 'modalidad' sea igual a '1'
creditos_2019_pachuca_modalidad_1 = creditos_2019_pachuca[creditos_2019_pachuca['modalidad'] == 1]
# Agrupar por mes y sumar los montos
total_creditos_2019_pachuca = creditos_2019_pachuca_modalidad_1.groupby('mes')['monto'].mean().reset_index()
total_creditos_2019_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2019_pachuca['año'] = 2019
total_creditos_2019_pachuca = total_creditos_2019_pachuca[['año', 'mes', 'montos']] # Reordenar las columnas

# Filtrar los datos donde 'modalidad' sea igual a '1'
creditos_2020_pachuca_modalidad_1 = creditos_2020_pachuca[creditos_2020_pachuca['modalidad'] == 1]
# Agrupar por mes y sumar los montos
total_creditos_2020_pachuca = creditos_2020_pachuca_modalidad_1.groupby('mes')['monto'].mean().reset_index()
total_creditos_2020_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2020_pachuca['año'] = 2020
total_creditos_2020_pachuca = total_creditos_2020_pachuca[['año', 'mes', 'montos']] # Reordenar las columnas

# Filtrar los datos donde 'modalidad' sea igual a '1'
creditos_2021_pachuca_modalidad_1 = creditos_2021_pachuca[creditos_2021_pachuca['modalidad'] == 1]
# Agrupar por mes y sumar los montos
total_creditos_2021_pachuca = creditos_2021_pachuca_modalidad_1.groupby('mes')['monto'].mean().reset_index()
total_creditos_2021_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2021_pachuca['año'] = 2021
total_creditos_2021_pachuca = total_creditos_2021_pachuca[['año', 'mes', 'montos']] # Reordenar las columnas

# Filtrar los datos donde 'modalidad' sea igual a '1'
creditos_2022_pachuca_modalidad_1 = creditos_2022_pachuca[creditos_2022_pachuca['modalidad'] == 1]
# Agrupar por mes y sumar los montos
total_creditos_2022_pachuca = creditos_2022_pachuca_modalidad_1.groupby('mes')['monto'].mean().reset_index()
total_creditos_2022_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2022_pachuca['año'] = 2022
total_creditos_2022_pachuca = total_creditos_2022_pachuca[['año', 'mes', 'montos']] # Reordenar las columnas

# Filtrar los datos donde 'modalidad' sea igual a '1'
creditos_2023_pachuca_modalidad_1 = creditos_2023_pachuca[creditos_2023_pachuca['modalidad'] == 1]
# Agrupar por mes y sumar los montos
total_creditos_2023_pachuca = creditos_2023_pachuca_modalidad_1.groupby('mes')['monto'].mean().reset_index()
total_creditos_2023_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2023_pachuca['año'] = 2023
total_creditos_2023_pachuca = total_creditos_2023_pachuca[['año', 'mes', 'montos']] # Reordenar las columnas

total_creditos_nueva_pachuca = pd.concat([total_creditos_2019_pachuca, total_creditos_2020_pachuca, total_creditos_2021_pachuca, total_creditos_2022_pachuca, total_creditos_2023_pachuca], ignore_index=True)
total_creditos_nueva_pachuca.head()

,año,mes,montos
0,2019,1,974821.397143
1,2019,2,843203.306250
2,2019,3,701951.655714
3,2019,4,747503.219000
4,2019,5,959885.878824


In [57]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np

# Preparar datos para el modelo
X = total_creditos_nueva_pachuca[['año', 'mes']]
y = total_creditos_nueva_pachuca['montos']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [58]:
from sklearn.metrics import r2_score
# Hacer predicciones en el conjunto de prueba
y_pred = modelo.predict(X_test)
# Calcular el coeficiente de determinación (R^2)
r2 = r2_score(y_test, y_pred)
print("Coeficiente de determinación (R^2):", r2)

from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
print("Error cuadrático medio (MSE):", mse)

from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)
print("Error absoluto medio (MAE):", mae)
rmse = np.sqrt(mse)
print("Raíz del error cuadrático medio (RMSE):", rmse)

Coeficiente de determinación (R^2): -0.2887998067105848
Error cuadrático medio (MSE): 182510542435.08823
Error absoluto medio (MAE): 287384.9808651063
Raíz del error cuadrático medio (RMSE): 427212.5260746555


In [59]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2023, 5),
                            'mes': range(8, 13)})

# DF con todas las combinaciones posibles de año y mes para 2024-2026
fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
                                  'mes': np.tile(range(1, 13), 3)})
nuevas_fechas_nueva = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas_nueva)
# Agregamos las predicciones al DataFrame de nuevas fechas
nuevas_fechas_nueva['montos'] = predicciones
total_nueva_pachuca = pd.concat([total_creditos_nueva_pachuca, nuevas_fechas_nueva], ignore_index=True)
total_nueva_pachuca = total_nueva_pachuca.groupby('año')['montos'].sum().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
total_nueva_pachuca['modalidad'] = 'Nueva'
# Función para formatear números con separadores de coma y dos decimales
def format_number(x):
   return '{:,.2f}'.format(x)
total_nueva_pachuca['montos'] = total_nueva_pachuca['montos'].map(format_number)
total_nueva_pachuca

,año,montos,modalidad
0,2019,"9,846,611.58",Nueva
1,2020,"12,753,289.06",Nueva
2,2021,"10,919,234.52",Nueva
3,2022,"11,692,083.11",Nueva
4,2023,"15,409,017.20",Nueva
5,2024,"13,978,296.94",Nueva
6,2025,"14,794,752.48",Nueva
7,2026,"15,611,208.01",Nueva


In [60]:
# Filtrar los datos donde 'modalidad' sea igual a '3'
creditos_2019_pachuca_modalidad_3 = creditos_2019_pachuca[creditos_2019_pachuca['modalidad'] == 3]
# Agrupar por mes y sumar los montos
total_creditos_2019_pachuca = creditos_2019_pachuca_modalidad_3.groupby('mes')['monto'].mean().reset_index()
total_creditos_2019_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2019_pachuca['año'] = 2019
total_creditos_2019_pachuca = total_creditos_2019_pachuca[['año', 'mes', 'montos']] # Reordenar las columnas

# Filtrar los datos donde 'modalidad' sea igual a '3'
creditos_2020_pachuca_modalidad_3 = creditos_2020_pachuca[creditos_2020_pachuca['modalidad'] == 3]
# Agrupar por mes y sumar los montos
total_creditos_2020_pachuca = creditos_2020_pachuca_modalidad_3.groupby('mes')['monto'].mean().reset_index()
total_creditos_2020_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2020_pachuca['año'] = 2020
total_creditos_2020_pachuca = total_creditos_2020_pachuca[['año', 'mes', 'montos']] # Reordenar las columnas

creditos_2021_pachuca_modalidad_3 = creditos_2021_pachuca[creditos_2021_pachuca['modalidad'] == 3]
# Agrupar por mes y sumar los montos
total_creditos_2021_pachuca = creditos_2021_pachuca_modalidad_3.groupby('mes')['monto'].mean().reset_index()
total_creditos_2021_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2021_pachuca['año'] = 2021
total_creditos_2021_pachuca = total_creditos_2021_pachuca[['año', 'mes', 'montos']] # Reordenar las columnas

creditos_2022_pachuca_modalidad_3 = creditos_2022_pachuca[creditos_2022_pachuca['modalidad'] == 3]
# Agrupar por mes y sumar los montos
total_creditos_2022_pachuca = creditos_2022_pachuca_modalidad_3.groupby('mes')['monto'].mean().reset_index()
total_creditos_2022_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2022_pachuca['año'] = 2022
total_creditos_2022_pachuca = total_creditos_2022_pachuca[['año', 'mes', 'montos']] # Reordenar las columnas

creditos_2023_pachuca_modalidad_3 = creditos_2023_pachuca[creditos_2023_pachuca['modalidad'] == 3]
# Agrupar por mes y sumar los montos
total_creditos_2023_pachuca = creditos_2023_pachuca_modalidad_3.groupby('mes')['monto'].mean().reset_index()
total_creditos_2023_pachuca.rename(columns={'monto': 'montos'}, inplace=True)
total_creditos_2023_pachuca['año'] = 2023
total_creditos_2023_pachuca = total_creditos_2023_pachuca[['año', 'mes', 'montos']] # Reordenar las columnas

total_creditos_usada_pachuca = pd.concat([total_creditos_2019_pachuca, total_creditos_2020_pachuca, total_creditos_2021_pachuca, total_creditos_2022_pachuca, total_creditos_2023_pachuca], ignore_index=True)
total_creditos_usada_pachuca.head()

,año,mes,montos
0,2019,1,704528.772857
1,2019,2,744683.769231
2,2019,3,652554.335333
3,2019,4,619765.764545
4,2019,5,746784.559333


In [61]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np
# Preparar datos para el modelo
X = total_creditos_usada_pachuca[['año', 'mes']]
y = total_creditos_usada_pachuca['montos']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [62]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2023, 5),
                            'mes': range(8, 13)})

# DF con todas las combinaciones posibles de año y mes para 2024-2026
fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
                                  'mes': np.tile(range(1, 13), 3)})
nuevas_fechas_usada = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas_usada)
# Agregamos las predicciones al DataFrame de nuevas fechas
nuevas_fechas_usada['montos'] = predicciones
total_usada_pachuca = pd.concat([total_creditos_usada_pachuca, nuevas_fechas_usada], ignore_index=True)
total_usada_pachuca = total_usada_pachuca.groupby('año')['montos'].sum().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
total_usada_pachuca['modalidad'] = 'Usada'
# Función para formatear números con separadores de coma y dos decimales
def format_number(x):
   return '{:,.2f}'.format(x)
total_usada_pachuca['montos'] = total_usada_pachuca['montos'].map(format_number)
total_usada_pachuca

,año,montos,modalidad
0,2019,"9,004,345.28",Usada
1,2020,"9,648,728.91",Usada
2,2021,"10,322,757.45",Usada
3,2022,"11,312,490.43",Usada
4,2023,"5,144,929.78",Usada
5,2024,"12,732,058.75",Usada
6,2025,"13,484,690.34",Usada
7,2026,"14,237,321.92",Usada


In [63]:
forecast_creditosgral_totales_pachuca = pd.concat([total_todos_pachuca, total_nueva_pachuca,total_usada_pachuca], ignore_index=True)
# Datos
trimestres = forecast_creditosgral_totales_pachuca['año']
modalidades = forecast_creditosgral_totales_pachuca['modalidad']
montos = forecast_creditosgral_totales_pachuca['montos']

# Limpiar las cadenas de montos y convertirlas a números
promedios_limpios = [float(montos.replace(',', '')) for montos in montos]

fig = go.Figure()
# Agregar líneas para cada modalidad
for modalidad in modalidades.unique():
    df_modalidad = forecast_creditosgral_totales_pachuca[forecast_creditosgral_totales_pachuca['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['año'],
        y=df_modalidad['montos'].apply(lambda x: float(x.replace(',', ''))),  # Usar los montos limpios
        mode='markers+lines',
        name=modalidad
    ))

fig.update_layout(
    title='Monto total anual',
    yaxis=dict(gridcolor='#dddcda'),
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

# Agregar etiquetas de texto a los puntos
for trace in fig.data:
    df_modalidad = forecast_creditosgral_totales_pachuca[forecast_creditosgral_totales_pachuca['modalidad'] == trace.name]
    for i, point in enumerate(trace.y):
        fig.add_annotation(
            x=trace.x[i], 
            y=point, 
            text=f'{df_modalidad.iloc[i]["montos"]}', 
            showarrow=False,
            font=dict(color='black', size=10),
            yshift=10
        )


# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, os.path.join(carpeta, f'{nombre_archivo}.html'))

# Exportar
guardar_grafico_como_html(fig, 'g_scatt_forecast_creditgral_promedio', carpeta='assets/graficas')
fig.show()

### DATOS DIRECTOS

In [64]:
data_2 = {
    'Año': [2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026],
    'Créditos (todos)': ['$640,236.65','$687,703.61',	'$779,936.30','$836,558.70','$836,558.70',	'$836,558.70',	'$836,558.70',	'$836,558.70'],
    'Vivienda Nueva': ['$831,399.91',	'$1,111,702.88',	'$1,044,401.67',	'$983,745.67',	'$983,745.67',	'$983,745.67',	'$983,745.67',	'$983,745.67'],
    'Vivienda Usada': ['$761,944.77',	'$778,140.36',	'$853,781.34',	'$970,042.24',	'$970,042.24',	'$970,042.24',	'$970,042.24',	'$970,042.24']
}
df_2 = pd.DataFrame(data_2)

# Reemplazar los símbolos y convertir a float
for col in df_2.columns:
    if col != 'Año':
        df_2[col] = df_2[col].str.replace('$', '').str.replace(',', '').astype(float)

# Reestructurar los datos
trimestres = []
modalidades = []
montos = []

for index, row in df_2.iterrows():
    for col in df_2.columns[1:]:  # Excluimos la columna de Año
        trimestres.append(row['Año'])
        modalidades.append(col)
        montos.append(row[col])

# Crear el nuevo DataFrame
nuevo_df_2 = pd.DataFrame({
    'trimestres': trimestres,
    'modalidades': modalidades,
    'montos': montos
})

In [65]:
import plotly.graph_objects as go
import os
import plotly.io as pio
import locale

# Configurar el locale para formatear los números como moneda
locale.setlocale(locale.LC_ALL, '')

# Datos
trimestres = nuevo_df_2['trimestres']
modalidades = nuevo_df_2['modalidades']
montos = nuevo_df_2['montos']

fig = go.Figure()

# Agregar líneas para cada modalidad
for modalidad in modalidades.unique():
    df_modalidad = nuevo_df_2[nuevo_df_2['modalidades'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['trimestres'],
        y=df_modalidad['montos'],
        mode='markers+lines',
        name=modalidad
    ))

fig.update_layout(
    title='Monto Promedio por Crédito',
    yaxis=dict(gridcolor='#dddcda'),
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

# Agregar etiquetas de texto a los puntos
for trace in fig.data:
    df_modalidad = nuevo_df_2[nuevo_df_2['modalidades'] == trace.name]
    for i, point in enumerate(trace.y):
        # Formatear el valor del monto como una cadena en formato de precio
        monto_formateado = locale.currency(df_modalidad.iloc[i]["montos"], grouping=True)
        fig.add_annotation(
            x=trace.x[i], 
            y=point, 
            text=f'{monto_formateado}', 
            showarrow=False,
            font=dict(color='black', size=10),
            yshift=10
        )


# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, os.path.join(carpeta, f'{nombre_archivo}.html'))

# Exportar
guardar_grafico_como_html(fig, 'g_scatt_forecast_creditgral_promedio_direct', carpeta='assets/graficas')
fig.show()
